In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [4]:
df = ratings.merge(movies, on='movieId')

df = df.drop(columns=['timestamp'])

print(df.isnull().sum())

print(df.head())

userId     0
movieId    0
rating     0
title      0
genres     0
dtype: int64
   userId  movieId  rating                        title  \
0       1        1     4.0             Toy Story (1995)   
1       1        3     4.0      Grumpier Old Men (1995)   
2       1        6     4.0                  Heat (1995)   
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


In [45]:
rating_mean = df.groupby('title').agg({'rating':'mean'}).reset_index()
rating_count = df.groupby('title').agg({'rating':'count'}).reset_index()

print(rating_mean.head())
print(rating_count.head())

                                     title  rating
0                               '71 (2014)     4.0
1  'Hellboy': The Seeds of Creation (2004)     4.0
2                   'Round Midnight (1986)     3.5
3                      'Salem's Lot (2004)     5.0
4                'Til There Was You (1997)     4.0
                                     title  rating
0                               '71 (2014)       1
1  'Hellboy': The Seeds of Creation (2004)       1
2                   'Round Midnight (1986)       2
3                      'Salem's Lot (2004)       1
4                'Til There Was You (1997)       2


In [79]:
rating_mean.rename(columns={'rating': 'rating_x'}, inplace=True)
rating_count.rename(columns={'rating': 'rating_y'}, inplace=True)

movies_mean_count = rating_mean.merge(rating_count, on='title', suffixes=['_x', '_y'])

movies_mean_count = movies_mean_count.copy()

df.loc[:, 'rating_x'] = movies_mean_count['rating_x']

df.loc[:, 'rating_y'] = movies_mean_count['rating_y']


In [80]:
df_unique = df.drop_duplicates(subset=['title'])

In [81]:
c = df['rating'].mean()
m = df['rating_y'].quantile(0.3)

qualified_movies = df_unique[df_unique['rating_y'] >= m]

print(df[['title', 'rating_y']].sort_values(by='rating_y', ascending=False).head(20))

qualified_movies = qualified_movies.copy()
qualified_movies.loc[:, "score"] = (
    (qualified_movies["rating_x"] * qualified_movies["rating_y"] + c * m) /
    (qualified_movies["rating_y"] + m)
)
qualified_movies = qualified_movies.sort_values(by='score', ascending=False)


                                            title  rating_y
6865                   Kicking & Screaming (2005)     307.0
4662                       This Is England (2006)     238.0
7421                     The Darkest Minds (2018)     220.0
405                   Sweet Hereafter, The (1997)     204.0
565                                Sabrina (1995)     201.0
3499                             Grown Ups (2010)     192.0
836                             Striptease (1996)     189.0
298                                Troll 2 (1990)     183.0
2940   Puppet Master III: Toulon's Revenge (1991)     181.0
8949                                Eragon (2006)     178.0
8982                             Road, The (2009)     177.0
744                             Wyatt Earp (1994)     171.0
3474                  Terminator Salvation (2009)     170.0
2139  Imaginarium of Doctor Parnassus, The (2009)     164.0
207                             Sister Act (1992)     161.0
5572              Boys of St. Vincent, T

In [82]:
print(qualified_movies[['title', 'score']].head(20))

                                                  title     score
8227                          Pump Up the Volume (1990)  4.471139
219                                    Gladiator (2000)  4.459595
752   City Slickers II: The Legend of Curly's Gold (...  4.449493
3672                             Midnight Cowboy (1969)  4.441498
954   Star Wars: Episode II - Attack of the Clones (...  4.432658
3861                                       Titus (1999)  4.432658
2209                                   Prisoners (2013)  4.412247
1641                                Blood Simple (1984)  4.369075
679                                Losing Isaiah (1995)  4.359595
1727                           Untouchables, The (1987)  4.324493
3953                                 Dirty Harry (1971)  4.319865
1556  Star Wars: Episode VII - The Force Awakens (2015)  4.299662
3499                                   Grown Ups (2010)  4.283927
2235                     Avengers: Age of Ultron (2015)  4.265991
3757      